In [ ]:
# @authors: Raj Vardhan

<h1> Explanation based detection of adversarial samples for Twitter</h1>
<br>
<p>
This notebook trains a classifier on the explanations of benign (class 0) and adversarial (class 1) samples.
The explanations of a sample are in the form of a binary feature vector (same dimension as sample) where top k indices with highest gradient are 1 and rest are 0

In [1]:
import tensorflow as tf 
import pickle
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from cleverhans.attacks import SaliencyMapMethod,FastGradientMethod,CarliniWagnerL2,DeepFool
from cleverhans.utils import other_classes, set_log_level
from cleverhans.utils import pair_visual, grid_visual, AccuracyReport
from cleverhans.utils_mnist import data_mnist
from cleverhans.utils_tf import model_train, model_eval, model_argmax
from cleverhans.utils_keras import KerasModelWrapper, cnn_model
from cleverhans.model import *

import pandas
import sklearn
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.layers import Dense, Dropout, Activation
import pickle
import operator
from keras.models import Sequential
from graphviz import Digraph

### Load data and model
import os
import pickle
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import tensorflow as tf
import pickle
import datetime
from sklearn.model_selection import train_test_split

RuntimeError: module compiled against API version 0xc but this version of numpy is 0xb

RuntimeError: module compiled against API version 0xc but this version of numpy is 0xb

Using TensorFlow backend.


In [4]:
seed = 7
np.random.seed(seed)

In [5]:
#Preprocessing
gradB = np.load('twitter_data//grads_sorted_ind_benign.npy')
gradAdv = np.load('twitter_data//grads_sorted_ind_adv.npy')

#initialize explanation matrices
eB = np.zeros(gradB.shape)
eAdv = np.zeros(gradAdv.shape)

#how many top indices should be considered?
thresh = 8

#set top indices to 1 in the overall feature vector (same dimension as input samples)
for ind in range(gradB.shape[0]):
    eB[ind][ gradB[ind][0:thresh] ] = 1

for ind in range(gradAdv.shape[0]):
    eAdv[ind][ gradAdv[ind][0:thresh] ] = 1
    
no_of_col = eB.shape[1]
print('no. of columns is {}'.format(no_of_col))


# Prepare X and Y for training and testing
X_orig = np.vstack((eB, eAdv))

yB = np.zeros((eB.shape[0], 1))
yAdv = np.ones((eAdv.shape[0], 1))

Y_orig = np.vstack((yB, yAdv))

#shuffle the order
indices = np.random.permutation(X_orig.shape[0])
X = X_orig[indices]
Y = Y_orig[indices]

no. of columns is 28


In [15]:
#Train test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [19]:
print('x_train shape : {}'.format(X_train.shape))
print('y_train shape : {}'.format(y_train.shape))
print('x_test shape : {}'.format(X_test.shape))
print('y_test shape : {}'.format(y_test.shape))

x_train shape : (27798, 28)
y_train shape : (27798, 1)
x_test shape : (13692, 28)
y_test shape : (13692, 1)


In [7]:
'''
# All indices where the class is Malicious
ind=np.where(Y==1)

xM=X[ind,:]

# xM is malware samples
pickle.dump(xM,open(r"xmW_twitter.pickle","wb"))
'''

'\n# All indices where the class is Malicious\nind=np.where(Y==1)\n\nxM=X[ind,:]\n\n# xM is malware samples\npickle.dump(xM,open(r"xmW_twitter.pickle","wb"))\n'

In [20]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

num_hidden_1 = 20
num_hidden_2 = 20

# build DNNs
DNNmodel = Sequential()
DNNmodel.add(Dense(num_hidden_1, input_dim = no_of_col, kernel_initializer="uniform",activation="relu"))
DNNmodel.add(Dropout(0.5))
DNNmodel.add(Dense(num_hidden_1, kernel_initializer="uniform", activation="relu"))
DNNmodel.add(Dropout(0.5))
DNNmodel.add(Dense(2))
DNNmodel.add(Activation("softmax"))

In [21]:
#compile Model
DNNmodel.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# 3-fold validations
history=DNNmodel.fit(X_train, dummy_y,validation_split=0.33,
          epochs=100, batch_size=256)
#DNNmodel.summary()

#evaluation
score = DNNmodel.evaluate(X_train, dummy_y, batch_size=256)
print("Test Accuracy",score)
print("\n%s: %.2f%%" % (DNNmodel.metrics_names[1], score[1]*100))

#store the model weights and bias
W=DNNmodel.get_weights()
pickle.dump(W,open(r"W_exp_drebin.pickle","wb"))

Train on 18624 samples, validate on 9174 samples
Epoch 1/100
18624/18624 [==============================] - 1s 38us/step - loss: 0.5480 - acc: 0.8112 - val_loss: 0.3555 - val_acc: 0.8903
Epoch 2/100
18624/18624 [==============================] - 0s 11us/step - loss: 0.3246 - acc: 0.8970 - val_loss: 0.2301 - val_acc: 0.9293
Epoch 3/100
18624/18624 [==============================] - 0s 10us/step - loss: 0.2538 - acc: 0.9238 - val_loss: 0.2069 - val_acc: 0.9324
Epoch 4/100
18624/18624 [==============================] - 0s 10us/step - loss: 0.2392 - acc: 0.9316 - val_loss: 0.1968 - val_acc: 0.9382
Epoch 5/100
18624/18624 [==============================] - 0s 10us/step - loss: 0.2298 - acc: 0.9356 - val_loss: 0.1927 - val_acc: 0.9392
Epoch 6/100
18624/18624 [==============================] - 0s 12us/step - loss: 0.2244 - acc: 0.9370 - val_loss: 0.1898 - val_acc: 0.9402
Epoch 7/100
18624/18624 [==============================] - 0s 10us/step - loss: 0.2220 - acc: 0.9360 - val_loss: 0.1871 - v

Epoch 60/100
18624/18624 [==============================] - 0s 10us/step - loss: 0.1988 - acc: 0.9399 - val_loss: 0.1762 - val_acc: 0.9416
Epoch 61/100
18624/18624 [==============================] - 0s 9us/step - loss: 0.2017 - acc: 0.9381 - val_loss: 0.1742 - val_acc: 0.9447
Epoch 62/100
18624/18624 [==============================] - 0s 10us/step - loss: 0.2011 - acc: 0.9383 - val_loss: 0.1742 - val_acc: 0.9447
Epoch 63/100
18624/18624 [==============================] - 0s 9us/step - loss: 0.2006 - acc: 0.9387 - val_loss: 0.1750 - val_acc: 0.9443
Epoch 64/100
18624/18624 [==============================] - 0s 10us/step - loss: 0.2032 - acc: 0.9394 - val_loss: 0.1759 - val_acc: 0.9460
Epoch 65/100
18624/18624 [==============================] - 0s 9us/step - loss: 0.2002 - acc: 0.9396 - val_loss: 0.1756 - val_acc: 0.9454
Epoch 66/100
18624/18624 [==============================] - 0s 9us/step - loss: 0.2016 - acc: 0.9380 - val_loss: 0.1767 - val_acc: 0.9460
Epoch 67/100
18624/18624 [=====

In [ ]:
#thresh=15 acc: 88.67%
#thres=12  acc: 90.05%
#thresh=8  acc: 94.75%

In [42]:
y_pred = DNNmodel.predict(X_test)
y_pred_am = np.argmax(y_pred, axis=1)

tp=0
fp=0
tn=0
fn=0

for i in range(len(y_pred_am)):
    if y_test[i][0] == 1:
        if y_pred_am[i] == y_test[i]:
            tp = tp+1
        else:
            fn = fn+1
    
    elif y_test[i][0] == 0:
        if y_pred_am[i] == y_test[i]:
            tn = tn+1
        else:
            fp = fp+1

In [46]:
print('false positives: {} out of {} possible negatives (benign)'.format(fp, fp+tn))
print('false negatives: {} out of {} possible positives (adversarial)'.format(fn, fn+tp))
print('precision: {}%'.format(tp*100.0/(tp+fp)))
print('recall: {}%'.format(tp*100.0/(tp+fn)))

false positives: 390 out of 6405 possible negatives (benign)
false negatives: 341 out of 7287 possible positives (adversarial)
precision: 94.6837513631%
recall: 95.320433649%
